### not complete : dimention setting 
- refer to https://qiita.com/masahikoofjoyto/items/5f6c14c4d83eff5c596d
- Apply above site method to Get stater on TFv2 

In [14]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

In [15]:
# Download the file
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

path_to_file = os.path.dirname(path_to_zip)+"/spa-eng/spa.txt"
print("Download to..:",path_to_file)

Download to..: /Users/macico/.keras/datasets/spa-eng/spa.txt


In [16]:
#### Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                   if unicodedata.category(c) != 'Mn')

####  sentence preprocesser
def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # モデルが予測をいつ開始し、いつ終了すれば良いかを知らせるため
    w = '<start> ' + w + ' <end>'
    return w

### Tokenize ### 
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
    return tensor, lang_tokenizer


#ベクトル*文章数のテンソルのうち、最大次元数を返却
def max_length(tensor):
    return max(len(t) for t in tensor)

In [17]:
#### load nad tokenize ####
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    return zip(*word_pairs)

#### Make dataset ####
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)

    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [18]:
# Try experimenting with the size of that dataset
num_examples = 3000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = \
    max_length(target_tensor), max_length(input_tensor)

# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = \
    train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

2400 2400 600 600


In [20]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
#埋め込み次元
embedding_dim = 256
#En/decoderにおけるユニット数
units = 1024
# 辞書サイズ,
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

#文字列の配列をスライスすると、文字列のデータセット
#シャッフルで
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
#バッチサイズに区切る
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
@tf.function
def tf_encode(token,label):
    return tf.py_function(dataset_mapper, [token,label], [tf.int64, tf.int64])


In [39]:
X = tf.keras.Input(shape=(None,), batch_size=BATCH_SIZE)
embedded = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)(X)
lstm = tf.keras.layers.Bidirectional(
      tf.keras.layers.LSTM(embedding_dim, dropout=0.4, recurrent_dropout=0.4))(embedded)
fully_connected = tf.keras.layers.Dense(units=256, activation='relu')(lstm)
Y = tf.keras.layers.Dense(1, activation='softmax',name='final_layer')(fully_connected)

model = tf.keras.Model(inputs=X, outputs=Y)

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [41]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(64, None)]              0         
_________________________________________________________________
embedding_7 (Embedding)      (64, None, 256)           488704    
_________________________________________________________________
bidirectional_7 (Bidirection (64, 512)                 1050624   
_________________________________________________________________
dense_19 (Dense)             (64, 256)                 131328    
_________________________________________________________________
final_layer (Dense)          (64, 1)                   257       
Total params: 1,670,913
Trainable params: 1,670,913
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(dataset,epochs=5)

Train for 37 steps
Epoch 1/5
 9/37 [======>.......................] - ETA: 13s - loss: -575.4391 - accuracy: 0.1250

KeyboardInterrupt: 

In [23]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))
        attention_weights = tf.nn.softmax(self.V(score), axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [24]:
#  adamにする
optimizer = tf.keras.optimizers.Adam(1e-4)

In [77]:
def create_model():
    X = tf.keras.Input(shape=(None,), batch_size=BATCH_SIZE)
    embedded = tf.keras.layers.Embedding(vocab_inp_size, embedding_dim)(X)
    lstm, forward_h, forward_c, backward_h, backward_c = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            embedding_dim,
            return_sequences = True,
            return_state = True,
            dropout = 0.4,
            recurrent_dropout =0.4
        )
    )(embedded)
    state_h = tf.keras.layers.Concatenate()([forward_h,backward_h])
    context, attention_weights = Attention(embedding_dim)(lstm, state_h)

    fully_connected =tf.keras.layers.Dense(
        units = embedding_dim,
        activation = 'relu'
    )(context)
    
    Y = tf.keras.layers.Dense(
        1, 
        name='final_layer'
    )(fully_connected)

    model = tf.keras.Model(inputs=X, outputs=Y)

    model.compile(
        loss = loss_function,
        optimizer=tf.keras.optimizers.Adam(1e-4),
        metrics=['accuracy']
    )
    
    return model


In [74]:
# Computes the crossentropy loss between the labels and predictions.
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

#mean
def loss_function(real, pred):
    # element wiseで０でないかを判定
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    # paddingした部分を入力としないように位置をmaskingする
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [75]:
model = create_model()
model.summary()

1909
Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (64, None, 256)      488704      input_16[0][0]                   
__________________________________________________________________________________________________
bidirectional_15 (Bidirectional [(64, None, 512), (N 1050624     embedding_15[0][0]               
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (None, 512)          0           bidirectional_15[0][1]           
                                                                 bidirectional_15[0][3

In [76]:
model = create_model()
model.fit(dataset,epochs=10)

1909
Train for 37 steps
Epoch 1/10
 1/37 [..............................] - ETA: 20s

ValueError: Shape mismatch: The shape of labels (received (512,)) should equal the shape of logits except for the last dimension (received (64, 1)).